# Notebook for visualizing trajectories for both cities

In [25]:
import os
import sys

def find_project_root(target_folder="masteroppgave"):
    """Find the absolute path of a folder (e.g., 'masteroppgave') by searching upward."""
    currentdir = os.path.abspath("__file__")  # Get absolute script path
    while True:
        if os.path.basename(currentdir) == target_folder:
            return currentdir  # Found the target folder
        parentdir = os.path.dirname(currentdir)
        if parentdir == currentdir:  # Stop at filesystem root
            return None
        currentdir = parentdir  # Move one level up

# Find and append 'masteroppgave' to sys.path dynamically
project_root = find_project_root("masteroppgave")

if project_root:
    sys.path.append(project_root)
    print(f"Project root found: {project_root}")
else:
    raise RuntimeError("Could not find 'masteroppgave' directory")

Project root found: /Users/thomasnitsche/Developer/skole/master/masteroppgave


In [26]:
import folium
from folium.plugins import FeatureGroupSubGroup
import random
from constants import  *
from utils.helpers import file_handler
import pprint

## ROME

In [27]:
ROME_OUTPUT_FOLDER_FULL = "../dataset/rome/output/" 

trajectories = file_handler.load_trajectory_files(["R_ABU.txt", "R_ARU.txt",  "R_ECN.txt"], ROME_OUTPUT_FOLDER_FULL) ## Reads a list of trajectory files, return a dictionary with the filename as key and coordinates as values

In [28]:
import folium
import random

# Rome coordinate bounds
R_MAX_LON = 12.53
R_MIN_LON = 12.44
R_MAX_LAT = 41.93
R_MIN_LAT = 41.88

def visualize_trajectories_with_filtering(trajectories):
    """
    Visualizes multiple trajectories on a Folium map with checkboxes to filter them.

    Parameters:
    - trajectories (dict): Dictionary where keys are trajectory names, and values are lists of (lat, lon) coordinates.

    Returns:
    - A Folium map object.
    """

    if not trajectories:
        raise ValueError("Trajectory dictionary is empty!")

    first_trajectory = list(trajectories.values())[0]
    center_lat, center_lon = first_trajectory[0]  # Center the map

    # Initialize Folium map
    map_trajectories = folium.Map(location=[center_lat, center_lon], zoom_start=14)

    # Add bounding box for Rome coordinates
    folium.Rectangle(
        bounds=[(R_MIN_LAT, R_MIN_LON), (R_MAX_LAT, R_MAX_LON)],
        color="black",
        weight=2,
        fill=True,
        fill_opacity=0.1,
        popup="Rome Bounding Box"
    ).add_to(map_trajectories)

    # Define a set of distinct colors for trajectories
    traj_colors = [
        "red", "blue", "green", "purple", "orange", "pink", "brown", "cyan", "magenta", "yellow", "lime"
    ]
    random.shuffle(traj_colors)  # Shuffle colors for randomness

    # Add a FeatureGroup for each trajectory to enable toggling
    for idx, (traj_name, coords) in enumerate(trajectories.items()):
        traj_color = traj_colors[idx % len(traj_colors)]  # Assign a unique color
        traj_layer = folium.FeatureGroup(name=f"Trajectory: {traj_name}")  # Create a feature group

        # Add trajectory line to its feature group
        folium.PolyLine(
            coords,
            color=traj_color,
            weight=6.5,
            opacity=1,
            popup=f"Trajectory: {traj_name}",
        ).add_to(traj_layer)

        # Add markers for each point along the trajectory
        for lat, lon in coords:
            folium.CircleMarker(
                location=(lat, lon),
                radius=2,  # Bigger markers
                color="black",
                fill=True,
                fill_color="red",
                fill_opacity=1,
                popup=f"Point: ({lat:.5f}, {lon:.5f})"
            ).add_to(traj_layer)

        traj_layer.add_to(map_trajectories)  # Add trajectory layer to map

    # Add LayerControl to enable checkboxes
    folium.LayerControl(collapsed=False).add_to(map_trajectories)

    return map_trajectories

# Generate the map with filterable checkboxes
map_trajectories = visualize_trajectories_with_filtering(trajectories)

# Save the map
map_trajectories.save("trajectories_rome.html")

print("Map with filterable trajectories saved as trajectories_rome_filtered.html")


Map with filterable trajectories saved as trajectories_rome_filtered.html


## Porto


In [29]:
PORTO_OUTPUT_FOLDER_FULL = "../dataset/porto/output/" 

trajectories = file_handler.load_trajectory_files(["P_ABU.txt", "P_ARU.txt",  "P_ECN.txt"], PORTO_OUTPUT_FOLDER_FULL) ## Reads a list of trajectory files, return a dictionary with the filename as key and coordinates as values

In [30]:
import folium
import random

# Porto coordinate bounds
P_MAX_LON = -8.57
P_MIN_LON = -8.66
P_MAX_LAT = 41.19
P_MIN_LAT = 41.14

def visualize_trajectories_with_filtering(trajectories):
    """
    Visualizes multiple trajectories on a Folium map with checkboxes to filter them.

    Parameters:
    - trajectories (dict): Dictionary where keys are trajectory names, and values are lists of (lat, lon) coordinates.

    Returns:
    - A Folium map object.
    """

    if not trajectories:
        raise ValueError("Trajectory dictionary is empty!")

    first_trajectory = list(trajectories.values())[0]
    center_lat, center_lon = first_trajectory[0]  # Center the map

    # Initialize Folium map
    map_trajectories = folium.Map(location=[center_lat, center_lon], zoom_start=14)

    # Add bounding box for Porto coordinates
    folium.Rectangle(
        bounds=[(P_MIN_LAT, P_MIN_LON), (P_MAX_LAT, P_MAX_LON)],
        color="black",
        weight=2,
        fill=True,
        fill_opacity=0.1,
        popup="Porto Bounding Box"
    ).add_to(map_trajectories)

    # Define a set of distinct colors for trajectories
    traj_colors = [
        "red", "blue", "lime", "cyan", "magenta", "yellow", "orange", "purple", "white"
    ]
    random.shuffle(traj_colors)  # Shuffle colors for randomness

    # Add a FeatureGroup for each trajectory to enable toggling
    for idx, (traj_name, coords) in enumerate(trajectories.items()):
        traj_color = traj_colors[idx % len(traj_colors)]  # Assign a unique color
        traj_layer = folium.FeatureGroup(name=f"Trajectory: {traj_name}")  # Create a feature group

        # Add trajectory line to its feature group
        folium.PolyLine(
            coords,
            color=traj_color,
            weight=6.5,
            opacity=1,
            popup=f"Trajectory: {traj_name}",
        ).add_to(traj_layer)

        # Add markers for each point along the trajectory
        for lat, lon in coords:
            folium.CircleMarker(
                location=(lat, lon),
                radius=2,  # Bigger markers
                color="black",
                fill=True,
                fill_color="red",
                fill_opacity=1,
                popup=f"Point: ({lat:.5f}, {lon:.5f})"
            ).add_to(traj_layer)

        traj_layer.add_to(map_trajectories)  # Add trajectory layer to map

    # Add LayerControl to enable checkboxes
    folium.LayerControl(collapsed=False).add_to(map_trajectories)

    return map_trajectories

# Generate the map with filterable checkboxes
map_trajectories = visualize_trajectories_with_filtering(trajectories)

# Save the map
map_trajectories.save("trajectories_porto.html")

print("Map with filterable trajectories saved as trajectories_porto_filtered.html")


Map with filterable trajectories saved as trajectories_porto_filtered.html
